# 0. Imports

In [ ]:
%pip install flax
%pip install git+https://github.com/probml/rebayes.git
%pip install dynamax
%pip install optax
%pip install tqdm
%pip install jax-tqdm
%pip install augly
%pip install bayesian-optimization

In [ ]:
import jax.numpy as jnp
import jax.random as jr
import flax